In [ ]:
import util

train_ds,val_ds = util.get_train_val_set()

In [ ]:
for data in train_ds.take(1):
    print(data['image'].shape)

In [12]:
!rm -rf results* 


In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import main
import model
import util 
import pandas as pd

#exp 1 vgg_net r, resnet with regularizer
batch_sizes = [128, 256]
learning_rates = [0.001, 0.01]



# For vggnets 
all_results = []
train_set, val_set = util.get_train_val_set()
for pf,fn in [("vgg_lr",model.vggnet_l), ("vgg_sr",model.vggnet_s),("resr", model.resnet_model)]:
    results = main.run_experiment(fn,train_set,val_set,learning_rates=learning_rates,batch_sizes=batch_sizes,result_dir=f"./results_{pf}", patience=15)
    all_results.append(results)





L2 Reg <keras.regularizers.L2 object at 0x14999d923880>
Epoch 1/200
352/352 [==============================] - 5s 12ms/step - loss: 4.6780 - val_loss: 4.6776
Epoch 2/200
352/352 [==============================] - 3s 10ms/step - loss: 4.5617 - val_loss: 4.5018
Epoch 3/200
352/352 [==============================] - 3s 10ms/step - loss: 4.4559 - val_loss: 4.3059
Epoch 4/200
352/352 [==============================] - 3s 10ms/step - loss: 4.3469 - val_loss: 4.2341
Epoch 5/200
352/352 [==============================] - 3s 10ms/step - loss: 4.2563 - val_loss: 4.3802
Epoch 6/200
352/352 [==============================] - 3s 10ms/step - loss: 4.1532 - val_loss: 4.1540
Epoch 7/200
352/352 [==============================] - 3s 9ms/step - loss: 4.0768 - val_loss: 3.9801
Epoch 8/200
352/352 [==============================] - 3s 10ms/step - loss: 4.0157 - val_loss: 3.8642
Epoch 9/200
352/352 [==============================] - 3s 9ms/step - loss: 3.9428 - val_loss: 3.7999
Epoch 10/200
352/352 [======

INFO:tensorflow:Assets written to: ./results_vgg_lr/20241022160643/best.hd5/assets


L2 Reg <keras.regularizers.L2 object at 0x14999d923880>
Epoch 1/200
352/352 [==============================] - 4s 10ms/step - loss: 4.7545 - val_loss: 4.6687
Epoch 2/200
352/352 [==============================] - 3s 9ms/step - loss: 4.6424 - val_loss: 4.6257
Epoch 3/200
352/352 [==============================] - 3s 9ms/step - loss: 4.6187 - val_loss: 4.6141
Epoch 4/200
352/352 [==============================] - 3s 9ms/step - loss: 4.6119 - val_loss: 4.6105
Epoch 5/200
352/352 [==============================] - 3s 9ms/step - loss: 4.6097 - val_loss: 4.6093
Epoch 6/200
352/352 [==============================] - 3s 10ms/step - loss: 4.6090 - val_loss: 4.6089
Epoch 7/200
352/352 [==============================] - 3s 9ms/step - loss: 4.6087 - val_loss: 4.6088
Epoch 8/200
352/352 [==============================] - 3s 10ms/step - loss: 4.6086 - val_loss: 4.6087
Epoch 9/200
352/352 [==============================] - 3s 10ms/step - loss: 4.6086 - val_loss: 4.6087
Epoch 10/200
352/352 [=========

INFO:tensorflow:Assets written to: ./results_vgg_lr/20241022161141/best.hd5/assets


L2 Reg <keras.regularizers.L2 object at 0x14999d923880>
Epoch 1/200
176/176 [==============================] - 5s 21ms/step - loss: 4.7392 - val_loss: 4.6945
Epoch 2/200
176/176 [==============================] - 4s 20ms/step - loss: 4.6403 - val_loss: 4.6443
Epoch 3/200
176/176 [==============================] - 3s 19ms/step - loss: 4.5482 - val_loss: 4.4475
Epoch 4/200
176/176 [==============================] - 3s 20ms/step - loss: 4.4834 - val_loss: 4.4551
Epoch 5/200
176/176 [==============================] - 4s 20ms/step - loss: 4.3872 - val_loss: 4.2243
Epoch 6/200
176/176 [==============================] - 3s 19ms/step - loss: 4.2499 - val_loss: 4.0951
Epoch 7/200
176/176 [==============================] - 3s 20ms/step - loss: 4.1363 - val_loss: 4.1423
Epoch 8/200
176/176 [==============================] - 4s 20ms/step - loss: 4.0495 - val_loss: 3.8993
Epoch 9/200
176/176 [==============================] - 3s 19ms/step - loss: 3.9719 - val_loss: 3.8038
Epoch 10/200
176/176 [====

...

Epoch 1/200
352/352 [==============================] - 4s 10ms/step - loss: 4.5647 - val_loss: 4.3682
Epoch 2/200
352/352 [==============================] - 3s 10ms/step - loss: 4.2981 - val_loss: 4.2046
Epoch 3/200
352/352 [==============================] - 3s 10ms/step - loss: 4.1557 - val_loss: 4.0093
Epoch 4/200
352/352 [==============================] - 4s 10ms/step - loss: 4.0338 - val_loss: 3.9386
Epoch 5/200
352/352 [==============================] - 3s 10ms/step - loss: 3.9508 - val_loss: 3.7912
Epoch 6/200
352/352 [==============================] - 3s 10ms/step - loss: 3.8850 - val_loss: 3.7956
Epoch 7/200
352/352 [==============================] - 3s 10ms/step - loss: 3.8305 - val_loss: 3.7395
Epoch 8/200
352/352 [==============================] - 3s 9ms/step - loss: 3.7753 - val_loss: 3.7391
Epoch 9/200
352/352 [==============================] - 3s 10ms/step - loss: 3.7331 - val_loss: 3.6661
Epoch 10/200
352/352 [==============================] - 3s 10ms/step - loss: 3.6958

INFO:tensorflow:Assets written to: ./results_vgg_sr/20241022162136/best.hd5/assets


Epoch 1/200
352/352 [==============================] - 4s 10ms/step - loss: 4.6519 - val_loss: 4.7512
Epoch 2/200
352/352 [==============================] - 3s 10ms/step - loss: 4.5442 - val_loss: 4.5030
Epoch 3/200
352/352 [==============================] - 3s 9ms/step - loss: 4.5167 - val_loss: 4.4823
Epoch 4/200
352/352 [==============================] - 3s 9ms/step - loss: 4.5017 - val_loss: 4.4962
Epoch 5/200
352/352 [==============================] - 3s 10ms/step - loss: 4.4946 - val_loss: 4.4709
Epoch 6/200
352/352 [==============================] - 3s 9ms/step - loss: 4.4773 - val_loss: 4.4397
Epoch 7/200
352/352 [==============================] - 3s 10ms/step - loss: 4.4725 - val_loss: 4.3907
Epoch 8/200
352/352 [==============================] - 3s 10ms/step - loss: 4.4660 - val_loss: 4.4625
Epoch 9/200
352/352 [==============================] - 3s 10ms/step - loss: 4.4586 - val_loss: 4.4843
Epoch 10/200
352/352 [==============================] - 3s 10ms/step - loss: 4.4580 -

INFO:tensorflow:Assets written to: ./results_vgg_sr/20241022162508/best.hd5/assets


Epoch 1/200
176/176 [==============================] - 4s 21ms/step - loss: 4.6243 - val_loss: 4.6591
Epoch 2/200
176/176 [==============================] - 4s 20ms/step - loss: 4.3697 - val_loss: 4.4061
Epoch 3/200
176/176 [==============================] - 4s 20ms/step - loss: 4.2286 - val_loss: 4.1201
Epoch 4/200
176/176 [==============================] - 4s 21ms/step - loss: 4.1142 - val_loss: 4.0826
Epoch 5/200
176/176 [==============================] - 3s 20ms/step - loss: 4.0110 - val_loss: 4.0129
Epoch 6/200
176/176 [==============================] - 4s 21ms/step - loss: 3.9414 - val_loss: 3.8681
Epoch 7/200
176/176 [==============================] - 3s 20ms/step - loss: 3.8775 - val_loss: 3.9639
Epoch 8/200
176/176 [==============================] - 4s 20ms/step - loss: 3.8329 - val_loss: 3.6998
Epoch 9/200
176/176 [==============================] - 4s 20ms/step - loss: 3.7924 - val_loss: 3.7402
Epoch 10/200
176/176 [==============================] - 4s 21ms/step - loss: 3.754

In [14]:
all_results_df = pd.concat(all_results, ignore_index=False, sort=False)
all_results_df.to_csv("all_results_exp1.csv",index=False)

In [17]:
import main
import model
import util 
import pandas as pd

#exp 1 vgg_net no r, resnet with regularizer
batch_sizes = [128, 256]
learning_rates = [0.001, 0.01]



# For vggnets 
all_results = []
train_set, val_set = util.get_train_val_set()
for pf,fn in [("vgg_l",model.vggnet_l), ("vgg_s",model.vggnet_s),("res", model.resnet_model)]:
    results = main.run_experiment(fn,train_set,val_set,learning_rates=learning_rates,batch_sizes=batch_sizes,result_dir=f"./results_{pf}", patience=15)
    all_results.append(results)





L2 Reg None
Epoch 1/200
352/352 [==============================] - 5s 12ms/step - loss: 4.6269 - val_loss: 4.5921
Epoch 2/200
352/352 [==============================] - 3s 9ms/step - loss: 4.5209 - val_loss: 4.4463
Epoch 3/200
352/352 [==============================] - 3s 9ms/step - loss: 4.4425 - val_loss: 4.4146
Epoch 4/200
352/352 [==============================] - 3s 9ms/step - loss: 4.3799 - val_loss: 4.2717
Epoch 5/200
352/352 [==============================] - 3s 10ms/step - loss: 4.3042 - val_loss: 4.2722
Epoch 6/200
352/352 [==============================] - 3s 10ms/step - loss: 4.2189 - val_loss: 4.1295
Epoch 7/200
352/352 [==============================] - 3s 9ms/step - loss: 4.1439 - val_loss: 4.1239
Epoch 8/200
352/352 [==============================] - 3s 9ms/step - loss: 4.0542 - val_loss: 3.9067
Epoch 9/200
352/352 [==============================] - 3s 9ms/step - loss: 3.9775 - val_loss: 3.8741
Epoch 10/200
352/352 [==============================] - 3s 10ms/step - loss:

INFO:tensorflow:Assets written to: ./results_vgg_l/20241022170217/best.hd5/assets


L2 Reg None
Epoch 1/200
352/352 [==============================] - 4s 10ms/step - loss: 4.5967 - val_loss: 4.5141
Epoch 2/200
352/352 [==============================] - 3s 9ms/step - loss: 4.5531 - val_loss: 4.5305
Epoch 3/200
352/352 [==============================] - 3s 9ms/step - loss: 4.5306 - val_loss: 4.4715
Epoch 4/200
352/352 [==============================] - 3s 10ms/step - loss: 4.5221 - val_loss: 4.5788
Epoch 5/200
352/352 [==============================] - 3s 10ms/step - loss: 4.5129 - val_loss: 4.4967
Epoch 6/200
352/352 [==============================] - 3s 10ms/step - loss: 4.5089 - val_loss: 4.4615
Epoch 7/200
352/352 [==============================] - 3s 10ms/step - loss: 4.5075 - val_loss: 4.5321
Epoch 8/200
352/352 [==============================] - 3s 10ms/step - loss: 4.5041 - val_loss: 4.5357
Epoch 9/200
352/352 [==============================] - 3s 10ms/step - loss: 4.5016 - val_loss: 4.4917
Epoch 10/200
352/352 [==============================] - 3s 9ms/step - lo

INFO:tensorflow:Assets written to: ./results_vgg_l/20241022170824/best.hd5/assets


L2 Reg None
Epoch 1/200
176/176 [==============================] - 4s 20ms/step - loss: 4.6210 - val_loss: 4.6053
Epoch 2/200
176/176 [==============================] - 4s 20ms/step - loss: 4.4973 - val_loss: 4.5556
Epoch 3/200
176/176 [==============================] - 3s 19ms/step - loss: 4.4417 - val_loss: 4.4053
Epoch 4/200
176/176 [==============================] - 4s 20ms/step - loss: 4.3847 - val_loss: 4.3857
Epoch 5/200
176/176 [==============================] - 3s 20ms/step - loss: 4.3319 - val_loss: 4.2673
Epoch 6/200
176/176 [==============================] - 3s 20ms/step - loss: 4.2797 - val_loss: 4.2045
Epoch 7/200
176/176 [==============================] - 3s 20ms/step - loss: 4.2202 - val_loss: 4.1782
Epoch 8/200
176/176 [==============================] - 3s 20ms/step - loss: 4.1609 - val_loss: 4.1349
Epoch 9/200
176/176 [==============================] - 4s 20ms/step - loss: 4.1084 - val_loss: 4.0083
Epoch 10/200
176/176 [==============================] - 3s 19ms/step -

...

INFO:tensorflow:Assets written to: ./results_res/20241022173826/best.hd5/assets


Epoch 1/200
176/176 [==============================] - 4s 19ms/step - loss: 4.1026 - val_loss: 5.1896
Epoch 2/200
176/176 [==============================] - 3s 18ms/step - loss: 3.6557 - val_loss: 4.0562
Epoch 3/200
176/176 [==============================] - 3s 18ms/step - loss: 3.3848 - val_loss: 3.8068
Epoch 4/200
176/176 [==============================] - 3s 18ms/step - loss: 3.1568 - val_loss: 3.5033
Epoch 5/200
176/176 [==============================] - 3s 19ms/step - loss: 2.9699 - val_loss: 3.2625
Epoch 6/200
176/176 [==============================] - 3s 19ms/step - loss: 2.8264 - val_loss: 3.1053
Epoch 7/200
176/176 [==============================] - 3s 18ms/step - loss: 2.7070 - val_loss: 3.0607
Epoch 8/200
176/176 [==============================] - 3s 18ms/step - loss: 2.6123 - val_loss: 2.9904
Epoch 9/200
176/176 [==============================] - 3s 19ms/step - loss: 2.5248 - val_loss: 2.9193
Epoch 10/200
176/176 [==============================] - 3s 19ms/step - loss: 2.451

INFO:tensorflow:Assets written to: ./results_res/20241022174038/best.hd5/assets


Epoch 1/200
176/176 [==============================] - 4s 19ms/step - loss: 4.1298 - val_loss: 4.3521
Epoch 2/200
176/176 [==============================] - 3s 19ms/step - loss: 3.7093 - val_loss: 3.7547
Epoch 3/200
176/176 [==============================] - 3s 19ms/step - loss: 3.3722 - val_loss: 3.8826
Epoch 4/200
176/176 [==============================] - 3s 17ms/step - loss: 3.1068 - val_loss: 3.5118
Epoch 5/200
176/176 [==============================] - 3s 19ms/step - loss: 2.8830 - val_loss: 3.6587
Epoch 6/200
176/176 [==============================] - 3s 18ms/step - loss: 2.7025 - val_loss: 3.4992
Epoch 7/200
176/176 [==============================] - 3s 18ms/step - loss: 2.5523 - val_loss: 3.5122
Epoch 8/200
176/176 [==============================] - 3s 19ms/step - loss: 2.4355 - val_loss: 3.7148
Epoch 9/200
176/176 [==============================] - 3s 18ms/step - loss: 2.3289 - val_loss: 3.4251
Epoch 10/200
176/176 [==============================] - 3s 18ms/step - loss: 2.251

INFO:tensorflow:Assets written to: ./results_res/20241022174501/best.hd5/assets


In [18]:
all_results_df = pd.concat(all_results, ignore_index=False, sort=False)
all_results_df.to_csv("all_results_exp2.csv",index=False)